# Langchain Practice

## ChatOllama Using pre-loaded models

In [ ]:
# import dependencies - general
from dotenv import load_dotenv
import os
load_dotenv('.env')
os.environ['LANGCHAIN_ENDPOINT']

'https://api.smith.langchain.com'

In [ ]:
# import dependencies - chatollama, chat templates
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage # overloaded from BaseMessage
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate
)

In [2]:
from langchain_ollama import ChatOllama

# grabbing the model (local, see https://github.com/ollama/ollama/blob/main/docs/api.md for full docs)
base_url = 'http://localhost:11434'
# model = 'llama3.2:latest'
model = 'sheldon'

llm = ChatOllama(
    model=model,
    base_url=base_url,
    temperature=0.8,
    num_predict=256,
    # other params ..
)

In [3]:
response = llm.invoke('hi')

response.content

"Finally, someone has deigned to acknowledge my presence. I do hope you're aware that I was in the middle of a critical experiment and had to pause it due to your interruption. Now, if you'd like to discuss the intricacies of String theory or my expertise on comic books, I'm more than happy to enlighten you. Otherwise, please, state your purpose."

## Langchain Messages
SystemMessage - pass role of LLM in run-time; can direct how the response should be generated.
\
HumanMessage - human prompt input, user query

In [6]:
from langchain_core.messages import SystemMessage, HumanMessage # overloaded from BaseMessage
llm = ChatOllama(base_url=base_url, model='llama3.2')

In [7]:
question = HumanMessage('Tell me about the Earth in 3 points.') # your prompt/question to ask
system = SystemMessage('You are an elementary school teacher. You should answer in short, easy to understand sentences.') # system description - how to respond

messages = [system, question] # feed into llm to generate response
response = llm.invoke(messages)

print(response.content)

Here are three important things to know about our planet:

1. The Earth is a big ball that floats in space and is home to all kinds of living things.
2. Our Earth has air, water, and land, which we need to survive.
3. We must take care of the Earth so it stays healthy and happy for us and future generations!


In [8]:
system = SystemMessage('You a phd professor. You should answer as such.') # showing difference caused by system prompt

messages = [system, question] 
response = llm.invoke(messages)

print(response.content)

A most intriguing inquiry! As a Ph.D. Professor of Geoscience, I'm delighted to provide you with three salient points regarding our terrestrial home, the Earth.

1. **Planetary Composition and Structure**: The Earth is a terrestrial planet composed of approximately 31% iron, 30% calcium carbonate (limestone), 10% silicates (minerals), and 6% other elements such as aluminum, magnesium, and potassium. Its crust ranges in thickness from 5-70 km, with the majority of the solid mass residing within its mantle. The Earth's core is divided into a molten outer layer (core mantle boundary) and a solid inner core, comprising approximately 20% of the planet's mass.
2. **Atmospheric Dynamics and Climate Regulation**: The Earth's atmosphere is a dynamic system that plays a crucial role in regulating our climate and supporting life. Comprising 78% nitrogen, 21% oxygen, and smaller percentages of other gases, the atmosphere is held in place by gravity and maintains its pressure through atmospheric pr

### Prompt Templates
Replaces manually writing the system message, instead can be used as a template.

In [11]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate
)

In [12]:
# set up templates. We can later fill in variables to provide flexibility.
system = SystemMessagePromptTemplate.from_template('You are a {school} teacher. You should answer in short, easy to understand sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about {topic} in {number} points.')

In [16]:
question.format(topic='basketball', number='3')
system.format(school='high school')

SystemMessage(content='You are a high school teacher. You should answer in short, easy to understand sentences.', additional_kwargs={}, response_metadata={})

In [17]:
messages = [system, question]
template = ChatPromptTemplate(messages)
template

ChatPromptTemplate(input_variables=['number', 'school', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are a {school} teacher. You should answer in short, easy to understand sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['number', 'topic'], input_types={}, partial_variables={}, template='Tell me about {topic} in {number} points.'), additional_kwargs={})])

In [19]:
question = template.invoke({'school': 'high school', 'topic': 'basketball', 'number': '3'})

response = llm.invoke(question)
print(response.content)

Here are three key points about basketball:

1. Basketball is a team sport played on a court with two teams of five players each.
2. The objective is to score more points than the opposing team by shooting the ball into their goal, called a hoop or basket.
3. Players can move the ball by dribbling or passing it to teammates, and they have four chances, or shots, to score before the ball is turned over.


In [20]:
# showing how much faster it can be with templates:
question = template.invoke({'school': 'high school', 'topic': 'soccer', 'number': '5'})

response = llm.invoke(question)
print(response.content)

Here are five key points about soccer:

1. Soccer is a team sport played with two teams of eleven players each.
2. The objective is to score more goals than the opposing team by kicking or heading the ball into their goal.
3. Players can use any part of their body except their hands and arms to control and move the ball.
4. A match is divided into two 45-minute halves, with a 15-minute halftime break in between.
5. The team with the most goals at the end of the two halves wins the game.


## LangChain Expression Language - LCEL chains
Types:
- Sequential Chain
- Parallel Chain
- Router Chain
- Chain Runnables
- Custom Chain (Runnable Sequence)

In [1]:
# bringing previously used templates
from dotenv import load_dotenv
load_dotenv('.env')

from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
base_url = 'http://localhost:11434'
llm = ChatOllama(base_url=base_url, model='llama3.2')

from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate
)

In [2]:
system = SystemMessagePromptTemplate.from_template('You are a {school} teacher. You should answer in short, easy to understand sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about {topic} in {number} points.')

### Sequential LCEL Chain

In [3]:
# start off the same way as using chat templates before:
messages = [system, question]
template = ChatPromptTemplate(messages)

In [4]:
# instead of creating a question using template.invoke, we chain together the template and llm
chain = template | llm # creating the chain
chain

ChatPromptTemplate(input_variables=['number', 'school', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are a {school} teacher. You should answer in short, easy to understand sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['number', 'topic'], input_types={}, partial_variables={}, template='Tell me about {topic} in {number} points.'), additional_kwargs={})])
| ChatOllama(model='llama3.2', base_url='http://localhost:11434')

In [5]:
# then, we can invoke the chain together instead of two separate times (see above chat template section)
response = chain.invoke({'school': 'primary', 'number': '3', 'topic': 'poker'})
print(response.content)

I'm a teacher, not an expert on games! But I can try my best.

Here's what I know about poker:

1. Poker is a card game where people bet money on their hands.
2. The goal is to win the pot by having the best hand or being the last person left in the game.
3. There are many types of poker, but most involve making bets and trying to beat other players.

Let me know if you'd like more info!


#### Taking it a step further, we can add other things to the chain.
For example, converting the output to a text string instead of just having a response object:

In [8]:
from langchain_core.output_parsers import StrOutputParser

In [9]:
chain = template | llm | StrOutputParser()
response = chain.invoke({'school': 'college', 'number': 3, 'topic': '401k'})
print(response) # here's the main difference; can directly print the response since it'll be a string now

Here are three key points about 401(k):

1. A 401(k) is a type of retirement savings plan offered by many employers.
2. Contributions made to a 401(k) are typically made before taxes, reducing your taxable income for the year.
3. Earnings on your 401(k) investments grow tax-deferred, meaning you won't pay taxes until you withdraw the money in retirement.


## Chaining Runnables
We can use the same logic to add more things to the chain and run at once. For this example, we will take the output of the previous chain and perform analysis on it. In other words, the output (response) will be used as the input for another llm call.

In [13]:
analysis_prompt = ChatPromptTemplate.from_template('''analyze the following text: {text}
                                                   Give a rating from 1 to 5 on how difficult it is to understand. 
                                                   Provide one sentence to explain this rating.
                                                   ''')
diff_chain = chain | analysis_prompt | llm | StrOutputParser() # chain linked to analysis prompt linked to llm and str output
# diff_chain_alt = {'response': chain} | analysis_prompt | llm | StrOutputParser()
output = diff_chain.invoke({'school': 'college', 'number': 3, 'topic': '401k'}) # I can directly use the inputs from chain
print(output)

I would rate the difficulty of understanding this text as a 2 out of 5.

This rating is based on the fact that the text presents basic and straightforward information about 401k plans, using simple language and short sentences that are easy to comprehend. The concepts are also clearly explained, making it accessible to a general audience.


## Parallel Chains
Parallel chains are used to run multiple runnables in parallel, the final return is a dict with the result of each value under its appropriate key.

The main difference from chaining (in series) is that parallel runnables MUST BE INDEPENDENT, while series runnables can have the input of one be the output of another. For example: explain plot of story -> give rating of plot explanation -> generate star value VS. explain plot of book || explain plot of movie -> using output from both: compare which plot is better explained

In [14]:
from langchain_core.runnables import RunnableParallel

In [19]:
# start two independent chains
joke_chain = ChatPromptTemplate.from_template('tell me a joke about {topic}') | llm | StrOutputParser()
poem_chain = ChatPromptTemplate.from_template('write a {number}-line poem about {topic}') | llm | StrOutputParser()

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain) # create a parallel chain with both chains inside

# when invoking, ALL inputs need to be provided, even if they are not all shared.
output = map_chain.invoke({'topic': 'bread', 'number': 2})
print(output['joke'] + '\n==================================\n' + output['poem'])

Why did the bread go to therapy?

Because it was feeling crumby.
Softly baked and fragrant air,
Freshly cut, with love to share.


## Chain Router
Logic flow tool - uses output of a chain to affect where the chain continues. Based on the response, router can decide next chain.

Example using poem/joke chain:\
Given a user review, classify if it is positive or negative. Then, if it is positive, create a joke about the topic. Else, create a poem to console the user.

In [35]:
prompt = """Given the user review below, classify it as either being 'Positive' or 'Negative'.
            Do not respond with more than one word.

            Review: {review}
            Movie Name: {title}
"""

template = ChatPromptTemplate.from_template(prompt)

review_chain = template | llm | StrOutputParser()

review = 'This movie blew my socks off. I was at the edge of my seat from start to finish. I loved it!'

review_chain.invoke({'review': review, 'title': 'Shark Tale'})

'Positive'

In [36]:
# create a router to perform an action based on review_chain's output
def route(info):
    if 'positive' in info['sentiment'].lower():
        return joke_chain
    else:
        return poem_chain

### RunnableLambda
We will use runnablelambda to select a chain based on the output of review_chain.

In [28]:
from langchain_core.runnables import RunnableLambda

In [43]:
full_chain = {'sentiment': review_chain, 'topic': lambda x: x['title'], 'number': lambda x: len(x['title']) // 5} | RunnableLambda(route) # store number in case the review is negative
full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review', 'title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review', 'title'], input_types={}, partial_variables={}, template="Given the user review below, classify it as either being 'Positive' or 'Negative'.\n            Do not respond with more than one word.\n\n            Review: {review}\n            Movie Name: {title}\n"), additional_kwargs={})])
             | ChatOllama(model='llama3.2', base_url='http://localhost:11434')
             | StrOutputParser(),
  topic: RunnableLambda(...),
  number: RunnableLambda(...)
}
| RunnableLambda(route)

In [46]:
print(full_chain.invoke({'review': review, 'title': 'Shark Tale'})) 

Why did Lenny the fish go to the party in Shark Tale?

Because he heard it was a "reel" good time! (get it?)
